# Plot the emotions using the sliding window evaluation

In [ ]:
import matplotlib.pyplot as plt
import glob
import pandas as pd
import pickle
import numpy as np

emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
chosen_difference_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]


# Full aggregate per emotion

In [ ]:

# Plot full aggregate of emotions per emotion

score_filenames = glob.glob("data/results/*_roberta_wordwindow10_scores.pkl")

lens = []
for score_filename in score_filenames:
    with open(score_filename, 'rb') as f:
        scores = pickle.load(f)
        lens.append(len(scores["middle"]))

min_length = min(lens)
print(f"Score vector lengths: {lens}")
print(f"Shortest score vector: {min_length}")


# For each session, we map the scores to a fixed length

n_values = 100
if n_values > min_length:
    print(f"n_values ({n_values}) is longer than the shortest score vector ({min_length}), setting n_values to {min_length}")
    n_values = min_length
    
def map_to_fixed_length(scores, n_values):
    
    scores_per_value = len(scores) // n_values
    fixed_scores = [np.mean(scores[i*scores_per_value:(i+1)*scores_per_value], axis=0) for i in range(n_values)]
    fixed_scores = np.array(fixed_scores)
    return fixed_scores



emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
aggregated_scores = {
    "intro": np.zeros(28),
    "outro": np.zeros(28),
    "middle": np.zeros((n_values, 28)),
}

for score_filename in score_filenames:
    with open(score_filename, 'rb') as f:
        scores = pickle.load(f)

    scores["middle"] = map_to_fixed_length(scores["middle"], n_values)

    aggregated_scores["intro"] += scores["intro"]
    aggregated_scores["outro"] += scores["outro"]
    aggregated_scores["middle"] += scores["middle"]

aggregated_scores = {k: v / len(score_filenames) for k, v in aggregated_scores.items()}

# Smoothen the aggregated scores with numpy
smoothing_window = 10
smooth_aggregated_scores = {
    k: np.apply_along_axis(lambda m: np.convolve(m, np.ones(smoothing_window)/smoothing_window, mode='same'), 0, v)[smoothing_window//2:-smoothing_window//2]
    for k, v in aggregated_scores.items()
    }
#print(aggregated_scores)

plt.figure()
plt.title(f"Average emotion evolution over time")
for emotion in ["joy", "anger", "amusement", "sadness"]:
    plt.plot(aggregated_scores["middle"][:, emotions.index(emotion)], label=f"{emotion.capitalize()}")
plt.legend()
plt.savefig("data/results/plots/roberta_wordwindow10_all.png")
plt.show()

plt.figure()
plt.title(f"Average emotion evolution over time")
for emotion in ["joy", "anger", "amusement", "sadness"]:
    plt.plot(smooth_aggregated_scores["middle"][:, emotions.index(emotion)], label=f"{emotion.capitalize()}")
plt.legend()
plt.savefig("data/results/plots/roberta_wordwindow10_all_smoothed.png")
plt.show()

plt.figure()
plt.title(f"Average emotion evolution over time")
for emotion in chosen_difference_emotions:
    plt.plot(aggregated_scores["middle"][:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
plt.legend()
plt.savefig("data/results/plots/roberta_wordwindow10_chosen.png")
plt.show()

plt.figure()
plt.title(f"Average emotion evolution over time")
for emotion in chosen_difference_emotions:
    plt.plot(smooth_aggregated_scores["middle"][:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
plt.legend()
plt.savefig("data/results/plots/roberta_wordwindow10_chosen_smoothed.png")
plt.show()


In [ ]:
import seaborn as sns
from IPython.display import display
def compute_tables(aggregated_scores, title_prefix=""):
    emotion_average = {
        "Intro": aggregated_scores["intro"],
        "Gameplay": np.mean(aggregated_scores["middle"], axis=0),
        "Outro": aggregated_scores["outro"],
        "Total impact": aggregated_scores["outro"] - aggregated_scores["intro"],
        "Gameplay impact": np.mean(aggregated_scores["middle"], axis=0) - aggregated_scores["intro"],
        "Outro impact": aggregated_scores["outro"] - np.mean(aggregated_scores["middle"], axis=0),
    }
    df = pd.DataFrame.from_dict(emotion_average) * 100
    df.index = [e.capitalize() for e in emotions]
    pd.set_option('display.float_format', '{:.1f}'.format)
    display(df)

    style_cmap_diverging = 'vlag'
    style_cmap_sequential = 'Blues'

    full_scores_hm = df.style.background_gradient(cmap=style_cmap_diverging)
    display(full_scores_hm)

    with open(f"data/results/{title_prefix}full_scores.html", "w") as f:
        f.write(full_scores_hm.to_html())

    section_scores = df[["Intro", "Gameplay", "Outro"]]
    section_scores_hm = section_scores.style.background_gradient(cmap=style_cmap_sequential)
    html = section_scores_hm.to_html()

    with open(f"data/results/{title_prefix}section_scores.html", "w") as f:
        f.write(html)
    display(section_scores_hm)

    difference_scores = df[["Total impact", "Gameplay impact", "Outro impact"]]
    difference_scores_hm = difference_scores.style.background_gradient(cmap=style_cmap_diverging)
    display(difference_scores_hm)

    with open(f"data/results/{title_prefix}difference_scores.html", "w") as f:
        f.write(difference_scores_hm.to_html())

    chosen_section_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]
    section_scores_chosen = section_scores.filter(items=chosen_section_emotions, axis=0)
    section_scores_chosen_hm = section_scores_chosen.style.background_gradient(cmap=style_cmap_sequential)
    display(section_scores_chosen_hm)

    with open(f"data/results/{title_prefix}section_scores_chosen.html", "w") as f:
        f.write(section_scores_chosen_hm.to_html())

    chosen_difference_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]
    difference_scores_chosen = difference_scores.filter(items=chosen_difference_emotions, axis=0)
    difference_scores_chosen_hm = difference_scores_chosen.style.background_gradient(cmap=style_cmap_diverging)
    display(difference_scores_chosen_hm)

    with open(f"data/results/{title_prefix}difference_scores_chosen.html", "w") as f:
        f.write(difference_scores_chosen_hm.to_html())
compute_tables(aggregated_scores)

# Plot emotion per interacting NPC

In [ ]:
def compute_score_aggregate(files):
    aggregated_scores = {
        "intro": np.zeros(28),
        "outro": np.zeros(28),
        "middle": np.zeros((n_values, 28)),
    }
    for score_filename in files:
        with open(score_filename, 'rb') as f:
            scores = pickle.load(f)
        
        scores["middle"] = map_to_fixed_length(scores["middle"], n_values)

        aggregated_scores["intro"] += scores["intro"]
        aggregated_scores["outro"] += scores["outro"]
        aggregated_scores["middle"] += scores["middle"]

    aggregated_scores = {k: v / len(files) for k, v in aggregated_scores.items()}
    return aggregated_scores

In [ ]:
import matplotlib.pyplot as plt
import glob
# Plot full aggregate of emotions per emotion

emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']
filename = "./data/VG4R-Blackstories-convologs - Data.csv"
df = pd.read_csv(filename, sep=",")

NPC_emotions = []
score_filenames = glob.glob("data/results/*_roberta_wordwindow10_scores.pkl")
for score_filename in score_filenames:
    with open(score_filename, 'rb') as f:
        scores = pickle.load(f)
    if scores["middle_emotion"] not in NPC_emotions:
        NPC_emotions.append(scores["middle_emotion"])
print("Detected the following NPC emotions:", NPC_emotions)

aggregates_log = {}
for NPC_emotion in NPC_emotions:
    aggregated_scores = {
        "intro": np.zeros(28),
        "outro": np.zeros(28),
        "middle": np.zeros((n_values, 28)),
    }

    # Find the files that have the given emotion
    emotion_filenames = []
    for score_filename in score_filenames:
        with open(score_filename, 'rb') as f:
            scores = pickle.load(f)
        if scores["middle_emotion"] == NPC_emotion:
            emotion_filenames.append(score_filename)

    aggregated_scores = compute_score_aggregate(emotion_filenames)
    aggregates_log[NPC_emotion] = aggregated_scores

    plt.figure()
    plt.title(f"Evolution of emotions for {NPC_emotion.capitalize()} Amy")
    for emotion in chosen_difference_emotions:
        plt.plot(aggregated_scores["middle"][:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
    plt.legend()
    plt.savefig(f"data/results/plots/roberta_wordwindow10_{NPC_emotion}_chosen.png")
    plt.show()

    # Smoothen aggregates
    smoothing_window = 10
    smooth_aggregated_scores = {
        k: np.apply_along_axis(lambda m: np.convolve(m, np.ones(smoothing_window)/smoothing_window, mode='same'), 0, v)[smoothing_window//2:-smoothing_window//2]
        for k, v in aggregated_scores.items()
    }

    # plot it
    plt.figure()
    plt.title(f"Evolution of emotions for {NPC_emotion.capitalize()} Amy")
    for emotion in chosen_difference_emotions:
        plt.plot(smooth_aggregated_scores["middle"][:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
    plt.legend()
    plt.savefig(f"data/results/plots/roberta_wordwindow10_{NPC_emotion}_chosen_smoothed.png")
    plt.show()


In [ ]:
# Normalize scores
aggregates_log_mean = np.mean([aggregates_log[em]["middle"] for em in NPC_emotions], axis=0)
print(aggregates_log_mean.shape)

nms_perNPC = {}
for NPC_emotion in NPC_emotions:

    normalized_middle_scores_per_NPC = aggregates_log[NPC_emotion]["middle"] - aggregates_log_mean

    # Smoothen
    smoothing_window = 10
    smoothed_normalized_scores = np.apply_along_axis(lambda m: np.convolve(m, np.ones(smoothing_window)/smoothing_window, mode='same'), 0, normalized_middle_scores_per_NPC)[smoothing_window//2:-smoothing_window//2]

    nms_perNPC[NPC_emotion] = smoothed_normalized_scores

y_max = np.max([np.max(nms_perNPC[NPC_emotion]) for NPC_emotion in NPC_emotions])
y_min = np.min([np.min(nms_perNPC[NPC_emotion]) for NPC_emotion in NPC_emotions])

for NPC_emotion in NPC_emotions:

    smoothed_normalized_scores = nms_perNPC[NPC_emotion]

    # plot it
    plt.figure()
    plt.title(f"Evolution of emotions for {NPC_emotion.capitalize()} Amy")
    for emotion in chosen_difference_emotions:
        plt.plot(smoothed_normalized_scores[:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
    ax = plt.gca()
    ax.set_ylim([y_min, y_max])
    plt.legend()
    plt.savefig(f"data/results/plots/roberta_wordwindow10_{NPC_emotion}_chosen_smoothed_normalized.png")
    plt.show()

for NPC_emotion in NPC_emotions:
    if NPC_emotion == "neutral":
        continue

    netural_normalized_middle_scores = aggregates_log[NPC_emotion]["middle"] - aggregates_log["neutral"]["middle"]

    # Smoothen
    smoothing_window = 10
    smoothed_normalized_scores = np.apply_along_axis(lambda m: np.convolve(m, np.ones(smoothing_window)/smoothing_window, mode='same'), 0, netural_normalized_middle_scores)[smoothing_window//2:-smoothing_window//2]

    # plot it
    plt.figure()
    plt.title(f"Evolution of emotions for {NPC_emotion.capitalize()} Amy")
    for emotion in chosen_difference_emotions:
        plt.plot(smoothed_normalized_scores[:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
    plt.legend()
    plt.savefig(f"data/results/plots/roberta_wordwindow10_{NPC_emotion}_chosen_smoothed_neutralnormalized.png")
    plt.show()

In [ ]:
npc_emotion_average = {}
for NPC_emotion in NPC_emotions:
    npc_emotion_average[NPC_emotion.capitalize()] = aggregates_log[NPC_emotion]["middle"].mean(axis=0)
df = pd.DataFrame.from_dict(npc_emotion_average) * 100
df.index = [e.capitalize() for e in emotions]
pd.set_option('display.float_format', '{:.1f}'.format)

style_cmap_diverging = 'vlag'
style_cmap_sequential = 'Blues'

full_scores_horizontal = df.T
# Remove average from all columns
#full_scores_horizontal = full_scores_horizontal - full_scores_horizontal.mean(axis=0)
full_scores_hm = full_scores_horizontal.style.background_gradient(cmap=style_cmap_sequential)
display(full_scores_hm)

full_scores_hm = df.style.background_gradient(cmap=style_cmap_diverging)
display(full_scores_hm)

chosen_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]
emotion_scores_chosen = df.filter(items=chosen_emotions, axis=0)
emotion_scores_chosen_hm = emotion_scores_chosen.style.background_gradient(cmap=style_cmap_sequential)
display(emotion_scores_chosen_hm)

with open(f"data/results/npc_emotion_scores_chosen.html", "w") as f:
    f.write(emotion_scores_chosen_hm.to_html())


# Normalized tables
npc_emotion_average_normalized = {}
for NPC_emotion in NPC_emotions:
    npc_emotion_average_normalized[NPC_emotion.capitalize()] = aggregates_log[NPC_emotion]["middle"].mean(axis=0) - aggregates_log_mean.mean(axis=0)

df = pd.DataFrame.from_dict(npc_emotion_average_normalized) * 100
df.index = [e.capitalize() for e in emotions]
pd.set_option('display.float_format', '{:.1f}'.format)

style_cmap_diverging = 'vlag'
style_cmap_sequential = 'Blues'

full_scores_hm = df.style.background_gradient(cmap=style_cmap_diverging)
display(full_scores_hm)

chosen_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]
emotion_scores_chosen = df.filter(items=chosen_emotions, axis=0)

#def my_format(val):
#    return f"{val:,.2f}"
#dict_format = {e: my_format for e in chosen_emotions}

emotion_scores_chosen_hm = emotion_scores_chosen.style.background_gradient(cmap=style_cmap_sequential)
#emotion_scores_chosen_hm.format(dict_format)
display(emotion_scores_chosen_hm)

with open(f"data/results/npc_emotion_scores_chosen_normalized.html", "w") as f:
    f.write(emotion_scores_chosen_hm.to_html())

# Neutral normalized tables
npc_emotion_average_netural_normalized = {}
for NPC_emotion in NPC_emotions:
    if NPC_emotion == "neutral":
        continue
    npc_emotion_average_netural_normalized[NPC_emotion.capitalize()] = aggregates_log[NPC_emotion]["middle"].mean(axis=0) - aggregates_log["neutral"]["middle"].mean(axis=0)

df = pd.DataFrame.from_dict(npc_emotion_average_netural_normalized) * 100
df.index = [e.capitalize() for e in emotions]

style_cmap_diverging = 'vlag'
style_cmap_sequential = 'Blues'

full_scores_hm = df.style.background_gradient(cmap=style_cmap_diverging)
display(full_scores_hm)

chosen_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]
emotion_scores_chosen = df.filter(items=chosen_emotions, axis=0)
emotion_scores_chosen_hm = emotion_scores_chosen.style.background_gradient(cmap=style_cmap_sequential)
display(emotion_scores_chosen_hm)

with open(f"data/results/npc_emotion_scores_chosen_neutralnormalized.html", "w") as f:
    f.write(emotion_scores_chosen_hm.to_html())

In [ ]:
chosen_difference_emotions = ["Admiration", "Anger", "Approval", "Confusion", "Curiosity", "Excitement", "Gratitude", "Joy", "Neutral", "Sadness"]

for NPC_emotion in NPC_emotions:
    aggregated_scores = aggregates_log[NPC_emotion]
    compute_tables(aggregated_scores, title_prefix=f"{NPC_emotion}_")
    #plt.figure()
    #plt.title(f"Evolution of emotions for {NPC_emotion.capitalize()} Amy")
    #for emotion in chosen_difference_emotions:
    #    plt.plot(aggregated_scores["middle"][:, emotions.index(emotion.lower())], label=f"{emotion.capitalize()}")
    #plt.legend()
    #plt.show()

In [ ]:
import pickle

# For each npc plot positivity negativity neutrality
positive_emotions = ["admiration", "amusement", "approval", "caring", "desire", "excitement", "gratitude", "joy", "love", "optimism", "pride", "relief"]
negative_emotions = ["anger", "annoyance", "disappointment", "disapproval", "disgust", "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"]
neutral_emotions = ["confusion", "curiosity", "realization", "neutral", "surprise"]

scores_pos_neg_neutral = {}
for NPC_emotion in NPC_emotions:
    aggregated_scores = aggregates_log[NPC_emotion]
    # Compute positivity, negativity, neutrality
    positive_scores = aggregated_scores["middle"][:, [emotions.index(e) for e in positive_emotions]].mean(axis=1)
    negative_scores = aggregated_scores["middle"][:, [emotions.index(e) for e in negative_emotions]].mean(axis=1)
    neutral_scores = aggregated_scores["middle"][:, [emotions.index(e) for e in neutral_emotions]].mean(axis=1)
    scores_pos_neg_neutral[NPC_emotion] = {
        "Positive": positive_scores,
        "Negative": negative_scores,
        "Neutral": neutral_scores,
    }

    pickle.dump(scores_pos_neg_neutral, open("data/results/scores_pos_neg_neutral.pkl", "wb"))

    # Smoothen aggregates
    smoothing_window = 10
    smooth_aggregated_scores = {
        k: np.apply_along_axis(lambda m: np.convolve(m, np.ones(smoothing_window)/smoothing_window, mode='same'), 0, v)[smoothing_window//2:-smoothing_window//2]
        for k, v in scores_pos_neg_neutral[NPC_emotion].items()
    }

    plt.figure()
    plt.title(f"Evolution of emotions for {NPC_emotion.capitalize()} Amy")
    for emotion in ["Positive", "Negative", "Neutral"]:
        plt.plot(smooth_aggregated_scores[emotion], label=f"{emotion.capitalize()}")
    plt.legend()
    plt.savefig(f"data/results/plots/roberta_wordwindow10_{NPC_emotion}_pos_neg_neutral_smoothed.png")
    plt.show()